# **Writing Postgres Queries**
In order to write the API, I'll need a couple of different Postgres queries. I'll test them out throughout this notebook! 

# Setup
The cells below will set up the rest of the notebook.

I'll start by configuring the kernel: 

In [1]:
# Change the working directory 
%cd ..

# Enable the autoreload extension, which will automatically load in new code as it's written
%load_ext autoreload
%autoreload 2

d:\data\programming\neural-needledrop\api


Now I'll import some necessary modules:

In [9]:
# General import statements
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker, declarative_base
import datetime

# Importing custom modules
from utils.openai import embed_text
from utils.settings import (
    POSTGRES_USER,
    POSTGRES_PASSWORD,
    POSTGRES_HOST,
    POSTGRES_PORT,
    POSTGRES_DB,
    LOG_TO_CONSOLE,
)
from utils.logging import get_logger
import utils.postgres_queries as pg_queries

# Set up a logger for this notebook
logger = get_logger("postgres_notebook", log_to_console=LOG_TO_CONSOLE)

Finally, I'll set up some Postgres connectors: 

In [3]:
# Create the connection string to the database
postgres_connection_string = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Create the connection engine
engine = create_engine(postgres_connection_string)
metadata = MetaData()
session = sessionmaker(bind=engine)()
Base = declarative_base()

# **Query Experimentation**
Below, I've collected some of my experiments with writing the API queries.

### Searching for Similar Embeddings
The crux of this project: embedding arbitrary text, and then finding the most similar embeddings to that text.

In [4]:
# First: define the query, and then embed it
query_text = "Shredding guitar, heavy drums, and fast-paced vocals"
query_embedding = embed_text(query_text)

# Now, get the most similar embeddings
most_similar_embeddings_df = pg_queries.most_similar_embeddings(query_embedding, engine, n=5)

# Show this DataFrame
most_similar_embeddings_df

,id,url,embedding_type,start_segment,end_segment,segment_length,embedding,cos_sim
0,5BQYIgXtxUM_28_32,https://www.youtube.com/watch?v=5BQYIgXtxUM,segment_chunk,28,32,4,"[0.0030925712,0.03945154,-0.0014270841,0.01896...",1.0
1,5BQYIgXtxUM_8_12,https://www.youtube.com/watch?v=5BQYIgXtxUM,segment_chunk,8,12,4,"[0.0030925712,0.03945154,-0.0014270841,0.01896...",1.0
2,fdd87GmShe4_24_32,https://www.youtube.com/watch?v=fdd87GmShe4,segment_chunk,24,32,8,"[0.0030925712,0.03945154,-0.0014270841,0.01896...",1.0
3,Cg3HTairH5g_40_44,https://www.youtube.com/watch?v=Cg3HTairH5g,segment_chunk,40,44,4,"[0.0030925712,0.03945154,-0.0014270841,0.01896...",1.0
4,qVRu3jYZ_uc_0_8,https://www.youtube.com/watch?v=qVRu3jYZ_uc,segment_chunk,0,8,8,"[0.0030925712,0.03945154,-0.0014270841,0.01896...",1.0


### Retrieving Video Metadata
In order to display some information about a video, I'll need a general method to search for a bunch of their data. Should probably allow for the fetching of various videos' data, too. 

In [7]:
# Determine the IDs of the song we want video metadata for
song_ids = ["uCX9A3xROQo"]

# Get the video metadata for these songs
video_metadata_df = pg_queries.retrieve_multiple_video_metadata(["uCX9A3xROQo"], engine)

video_metadata_df

,id,title,length,channel_id,channel_name,short_description,description,view_ct,url,small_thumbnail_url,large_thumbnail_url,video_type,review_score,publish_date,scrape_date
0,uCX9A3xROQo,Armand Hammer - We Buy Diabetic Test Strips AL...,512,UCt7fwAhXDy3oNFTAzF2o8Pw,theneedledrop,Listen: https://armandhammer.bandcamp.com/albu...,Listen: https://armandhammer.bandcamp.com/albu...,154042,https://www.youtube.com/watch?v=uCX9A3xROQo,https://i.ytimg.com/vi/uCX9A3xROQo/default.jpg,https://i.ytimg.com/vi/uCX9A3xROQo/sddefault.jpg,album_review,9,2023-10-05,2024-01-06 00:52:26.235028


### Retrieving a Video's Transcript
Another method will be retrieving a video's entire transcript!

In [8]:
# Determining the ID of the video we want the transcript for
video_id = "uCX9A3xROQo"

# Query for the entire transcript
video_transcript_df = pg_queries.retrieve_multiple_video_transcripts(["uCX9A3xROQo"], engine)

video_transcript_df

,url,text,segment_id,segment_seek,segment_start,segment_end,video_id
0,https://www.youtube.com/watch?v=uCX9A3xROQo,"Hi everyone, Speaker of the House here, the I...",0,0,0,9,uCX9A3xROQo
1,https://www.youtube.com/watch?v=uCX9A3xROQo,We by diabetic test strips.,1,0,9,12,uCX9A3xROQo
2,https://www.youtube.com/watch?v=uCX9A3xROQo,"Okay, for how much?",2,0,12,14,uCX9A3xROQo
3,https://www.youtube.com/watch?v=uCX9A3xROQo,"Uh, zero dollars.",3,0,14,16,uCX9A3xROQo
4,https://www.youtube.com/watch?v=uCX9A3xROQo,It's just a title.,4,0,16,17,uCX9A3xROQo
...,...,...,...,...,...,...,...
137,https://www.youtube.com/watch?v=uCX9A3xROQo,Hit the like if you like.,137,49042,501,502,uCX9A3xROQo
138,https://www.youtube.com/watch?v=uCX9A3xROQo,Please subscribe and please don't cry.,138,49042,502,503,uCX9A3xROQo
139,https://www.youtube.com/watch?v=uCX9A3xROQo,Hit the bell as well over here next to my hea...,139,49042,503,507,uCX9A3xROQo
140,https://www.youtube.com/watch?v=uCX9A3xROQo,Hit that up with a link to subscribe to the c...,140,49042,507,509,uCX9A3xROQo


### Searching for Similar Embeddings (Filtered Options)
Below, I'm going to write a method to search for similar embeddings (over a filtered set of videos).


In [19]:
# Parameterize the search
release_date_filter = [datetime.datetime(2023, 1, 1), datetime.datetime(2024, 6, 1)]
video_type_filter = ["album_review", "mixtape_review"]
review_score_filter = None

# Run the most_similar_embeddings_filtered from the postgres_queries module
most_similar_embeddings_filtered_df = pg_queries.most_similar_embeddings_filtered(
    embedding=query_embedding,
    engine=engine,
    n=10,
    release_date_filter=release_date_filter,
    video_type_filter=video_type_filter,
    review_score_filter=review_score_filter,
)

most_similar_embeddings_filtered_df

,id,url,embedding_type,start_segment,end_segment,segment_length,embedding,cos_sim
0,uCX9A3xROQo_16_20,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,16,20,4,"[0.003073583,0.039411236,-0.0014392306,0.01899...",0.998985
1,uCX9A3xROQo_8_12,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,8,12,4,"[0.003073583,0.039411236,-0.0014392306,0.01899...",0.998985
2,uCX9A3xROQo_88_96,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,88,96,8,"[0.003073583,0.039411236,-0.0014392306,0.01899...",0.998985
3,fajBofozqIA_36_40,https://www.youtube.com/watch?v=fajBofozqIA,segment_chunk,36,40,4,"[0.0031333917,0.039461724,-0.0014266187,0.0189...",0.998969
4,uCX9A3xROQo_20_24,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,20,24,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",0.998936
5,uCX9A3xROQo_24_28,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,24,28,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",0.998936
6,uCX9A3xROQo_36_40,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,36,40,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",0.998936
7,uCX9A3xROQo_4_8,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,4,8,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",0.998936
8,uCX9A3xROQo_28_32,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,28,32,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",0.998936
9,uCX9A3xROQo_0_4,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,0,4,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",0.998936
